In [136]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pylab as plt

In [137]:
genotype_dict = {"dicer_ko": "KO", 
                 "dicer_overexpressed": "OE", 
                 "wild_type": "WT", 
                 "aaa_parental_stock": "parental_stock"}

In [138]:
# import mutations parental stock against parental stock consensus 
# parental stock consensus: was used to call mutations for the evolutionary samples

fname_parental = "all_mutations.EB_space.csv"

df_parental = pd.read_csv(fname_parental)
df_parental['genotype'] = 'P0'
df_parental['passage'] = "0"
df_parental['replicate'] = "0"

In [146]:
### import non-filtered mutations

# mutation call positions wrt to the EB reference 
fname_mutations_wild_type = '../../resources/run_workflow/results_cluster/wild_type/all_mutations.EB_space.csv'
fname_mutations_ko = '../../resources/run_workflow/results_cluster/dicer_KO/all_mutations.EB_space.csv'
fname_mutations_overexpressed = '../../resources/run_workflow/results_cluster/dicer_overexpression/all_mutations.EB_space.csv'


df_wild_type = pd.read_csv(fname_mutations_wild_type)
df_wild_type = df_wild_type.drop(['Unnamed: 0'], axis =1)
df_wild_type['genotype'] = 'wild_type'
df_wild_type['passage'] = df_wild_type['date']
df_wild_type['replicate'] = df_wild_type['patient']

df_ko = pd.read_csv(fname_mutations_ko)
df_ko = df_ko.drop(['Unnamed: 0'], axis =1)
df_ko['genotype'] = 'dicer_ko'
df_ko['passage'] = df_ko['date']
df_ko['replicate'] = df_ko['patient']

df_overexpressed = pd.read_csv(fname_mutations_overexpressed)
df_overexpressed = df_overexpressed.drop(['Unnamed: 0'], axis =1)
df_overexpressed['genotype'] = 'dicer_overexpressed'
df_overexpressed['passage'] = df_overexpressed['date']
df_overexpressed['replicate'] = df_overexpressed['patient']

# dataframe with all mutations from all samples
df = pd.concat([df_ko, df_overexpressed, df_wild_type])
df["genotype"] = df["genotype"].apply(lambda x: genotype_dict[x])


In [147]:
df = pd.concat([df, df_parental])

In [148]:
df = df.drop(['Unnamed: 0.1', 'sample', 'patient', 'date'], axis=1)

In [149]:
# add information 
# add information
df['Frq1'] = pd.to_numeric(df['Frq1'], errors='coerce')
df['Frq2'] = pd.to_numeric(df['Frq2'], errors='coerce')
df['Frq3'] = pd.to_numeric(df['Frq3'], errors='coerce')

df['n_reads_var'] = df['Rvar'] + df['Fvar']
df['coverage'] = df['Rtot'] + df['Ftot']
df['frequency'] = df['n_reads_var'] / df['coverage']
df['Frq_ave'] = df[['Frq1','Frq2','Frq3']].mean(axis=1)

df['passage'] = df['passage'].str.split('_').str[-1].astype('float')

df['position'] = df['Pos']

In [150]:
# basic filtering 

# Post-processing filtering of mutation calling

# filter out where Frq_ave == 0.0 
# that is something unexpected happening in ShoRAH which is due to the super high coverage
df = df[df['Frq_ave']!=0]

# strand bias test --> we dont run this one here
#df = df[df['Pval']>=0.05]

# minimum read support
minimum_read_support = 10 
df = df[df['n_reads_var']>=minimum_read_support]

In [151]:
df = df[[ 'genotype', 'passage', 'replicate','Pos', 'Ref', 'Var',  'Fvar', 'Rvar', 'Ftot', 'Rtot', 'Pval',
       'coverage', 'frequency']]

In [152]:
# filter for deletions
df = df[df['Var']=='-']

In [153]:
# here is the dataframe of the deletion calls

#df[df['Pos'].isin(list(range(5765,5774)) + list(range(276,284)))]
df_del = df[df['Pos'].isin([5765, 276])].sort_values(['genotype', 'passage', 'replicate', 'Pos'])

In [154]:
df_del.head()

,genotype,passage,replicate,Pos,Ref,Var,Fvar,Rvar,Ftot,Rtot,Pval,coverage,frequency
10519,KO,1.0,replicate_a,276,T,-,1605,1293,63841,36159,0.089223,100000,0.028980
11355,KO,1.0,replicate_a,5765,T,-,1868,2130,49674,46431,0.327744,96105,0.041600
20502,KO,1.0,replicate_b,276,T,-,676,544,23867,12960,0.064935,36827,0.033128
21455,KO,1.0,replicate_b,5765,T,-,529,653,15860,14357,0.139112,30217,0.039117
16454,KO,1.0,replicate_c,276,T,-,587,491,21638,12097,0.059347,33735,0.031955


In [155]:
# add dataframe of the quality scores

In [185]:
fname = 'WT_deletion_analysis.csv'
df_WT = pd.read_csv(fname)
df_WT['genotype']='WT'

fname = 'OE_deletion_analysis.csv'
df_OE = pd.read_csv(fname)
df_OE['genotype']='OE'

fname = 'KO_deletion_analysis.csv'
df_KO = pd.read_csv(fname)
df_KO['genotype']='KO'

fname = 'P0_deletion_analysis.csv'
df_P0 = pd.read_csv(fname)
df_P0['genotype']='P0'
df_P0['passage'] = "0"
df_P0['replicate'] = "0"

df_qual = pd.concat([df_WT, df_OE, df_KO, df_P0], ignore_index = True)
df_qual = df_qual[['del_in_region', 
                   'qual_del_in_region', 
                   'del_region_close_to_read_end',
                   'genotype', 
                   'replicate', 
                   'passage', 
                   'deletion_position']]
df_qual['del_in_region'] = df_qual['del_in_region'].astype(int)
df_qual['del_region_close_to_read_end'] = df_qual['del_region_close_to_read_end'].astype(int)
df_qual['passage'] = df_qual['passage'].str.split("_").str[1].astype(float)

In [186]:
df_qual['no_del_in_region'] = 1 - df_qual['del_in_region']
df_qual['qual_homo_region_with_del'] = df_qual['qual_del_in_region']*df_qual['del_in_region']
df_qual['qual_homo_region_no_del'] = df_qual['qual_del_in_region']*(1-df_qual['del_in_region'])
df_qual['homo_close_read_edge_with_del'] = df_qual['del_region_close_to_read_end']*df_qual['del_in_region']
df_qual['homo_close_read_edge_no_del'] = df_qual['del_region_close_to_read_end']*(1-df_qual['del_in_region'])

In [188]:
df_qual_del = pd.pivot_table(data= df_qual, 
                             values=['del_in_region', 
                                     'no_del_in_region',
                                     'qual_del_in_region', 
                                     'del_region_close_to_read_end', 
                                     'qual_homo_region_with_del', 
                                     'qual_homo_region_no_del',
                                     'homo_close_read_edge_with_del',
                                     'homo_close_read_edge_no_del'], 
                             index=['genotype', 'passage', 'replicate', 'deletion_position'], 
                             aggfunc={'del_in_region': np.sum, 
                                      'no_del_in_region': np.sum,
                                      'del_region_close_to_read_end': np.mean, 
                                      'qual_del_in_region': np.mean,
                                      'qual_homo_region_with_del': np.sum, 
                                      'qual_homo_region_no_del': np.sum,
                                      'homo_close_read_edge_with_del': np.sum,
                                      'homo_close_read_edge_no_del': np.sum,
                             }
                            )
df_qual_del = df_qual_del.reset_index()
df_qual_del['Pos'] = df_qual_del['deletion_position'].astype(float)

In [191]:
df_qual_del['mean_quality_reads_no_deletion'] = df_qual_del['qual_homo_region_no_del']/df_qual_del['no_del_in_region']
df_qual_del['mean_quality_reads_with_deletion'] = df_qual_del['qual_homo_region_with_del']/df_qual_del['del_in_region']


df_qual_del['proportion_reads_with_del_where_homopolymeric_region_close_read_edge'] = df_qual_del['homo_close_read_edge_with_del']/df_qual_del['del_in_region']
df_qual_del['proportion_reads_no_del_where_homopolymeric_region_close_read_edge'] = df_qual_del['homo_close_read_edge_no_del']/df_qual_del['no_del_in_region']


In [190]:
df_qual_del.head()

,genotype,passage,replicate,deletion_position,del_in_region,del_region_close_to_read_end,homo_close_read_edge_no_del,homo_close_read_edge_with_del,no_del_in_region,qual_del_in_region,qual_homo_region_no_del,qual_homo_region_with_del,Pos,mean_quality_reads_no_deletion,mean_quality_reads_with_deletion
0,KO,1.0,replicate_a,276,2931,0.166035,13990,169,82346,40.052065,3.297292e+06,118227.978771,276.0,1.435746,40.337079
1,KO,1.0,replicate_a,5765,4283,0.165486,12410,250,72219,39.395519,2.844628e+06,169207.780220,5765.0,2.342981,39.506836
2,KO,1.0,replicate_b,276,1237,0.164705,5144,90,30541,40.022827,1.221967e+06,49878.589744,276.0,1.633168,40.322223
3,KO,1.0,replicate_b,5765,1251,0.160827,3776,80,22725,39.380657,8.946816e+05,49509.025475,5765.0,2.178615,39.575560
4,KO,1.0,replicate_c,276,1088,0.155166,4474,84,28287,40.072340,1.133239e+06,43886.087912,276.0,1.551458,40.336478


In [193]:
df_result = pd.merge(df_qual_del, 
                     df_del, 
                     left_on=['genotype', 'passage', 'replicate', 'Pos'], 
                     right_on=['genotype', 'passage', 'replicate', 'Pos'])

In [194]:
df_result.columns

Index(['genotype', 'passage', 'replicate', 'deletion_position',
       'del_in_region', 'del_region_close_to_read_end',
       'homo_close_read_edge_no_del', 'homo_close_read_edge_with_del',
       'no_del_in_region', 'qual_del_in_region', 'qual_homo_region_no_del',
       'qual_homo_region_with_del', 'Pos', 'mean_quality_reads_no_deletion',
       'mean_quality_reads_with_deletion',
       'proportion_reads_with_del_where_homopolymeric_region_close_read_edge',
       'proportion_reads_no_del_where_homopolymeric_region_close_read_edge',
       'Ref', 'Var', 'Fvar', 'Rvar', 'Ftot', 'Rtot', 'Pval', 'coverage',
       'frequency'],
      dtype='object')

In [195]:
df_result[['genotype', 'passage', 'replicate', 'deletion_position','Fvar', 'Rvar', 'Ftot', 'Rtot', 
           'Pval', 'coverage', 'frequency', 'mean_quality_reads_no_deletion',
           'mean_quality_reads_with_deletion',
           'proportion_reads_with_del_where_homopolymeric_region_close_read_edge',
           'proportion_reads_no_del_where_homopolymeric_region_close_read_edge'
          ]].to_csv('Supplementary_Table_5.csv')